# Facility Location

A large supermarket chain in Quebec needs to build warehouses for a set of supermarkets it is opening in Laval. The locations of the supermarkets have been identified, but the locations of the warehouses have yet to be determined.

Several good candidate locations for the warehouses have been identified, but decisions must be made regarding 
how many warehouses to open and at which candidate locations to build them.

Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from the warehouse to the supermarket, and hence reduce the delivery cost. However, opening a warehouse has a fixed cost associated with it.

The goal is to find the optimal tradeoff between delivery costs and the costs of building new facilities.


Consider two supermarkets and nine warehouse candidates. The coordinates of each supermarket are provided in the following table.

| <i></i> | Coordinates |  
| --- | --- | 
| Supermarket 1 | (0,1.5) |
| Supermarket 2 | (2.5,1.2) |

The following table shows the coordinates of the candidate warehouse sites and the fixed cost of building the warehouse in millions of GBP.

| <i></i> | coordinates | fixed cost |
| --- | --- |  --- |
| Warehouse 1 | (0,0) | 3 |
| Warehouse 2 | (0,1) | 2 |
| Warehouse 3 | (0,2) | 3 |
| Warehouse 4 | (1,0) | 1 |
| Warehouse 5 | (1,1) | 3 | 
| Warehouse 6 | (1,2) | 3 |
| Warehouse 7 | (2,0) | 4 |
| Warehouse 8 | (2,1) | 3 |  
| Warehouse 9 | (2,2) | 2 |


The cost per mile is one million CAD.

## Python Implementation



In [1]:
from itertools import product
from math import sqrt

import gurobipy as gb
from gurobipy import *

# Data
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1
wh=['warehouse1','warehouse2','warehouse3','warehouse4','warehouse5','warehouse6','warehouse7','warehouse8','warehouse9']
sm=['supermarket1', 'supermarket2']

We define a function that determines the Euclidean distance between each facility and customer sites. 

In [2]:
# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

### Model 


In [3]:
shipping_cost

{(0, 0): 1.5,
 (0, 1): 0.5,
 (0, 2): 0.5,
 (0, 3): 1.8027756377319946,
 (0, 4): 1.118033988749895,
 (0, 5): 1.118033988749895,
 (0, 6): 2.5,
 (0, 7): 2.0615528128088303,
 (0, 8): 2.0615528128088303,
 (1, 0): 2.773084924772409,
 (1, 1): 2.5079872407968904,
 (1, 2): 2.6248809496813377,
 (1, 3): 1.9209372712298547,
 (1, 4): 1.5132745950421556,
 (1, 5): 1.7,
 (1, 6): 1.3,
 (1, 7): 0.5385164807134504,
 (1, 8): 0.9433981132056605}

In [4]:
prob=gb.Model('Facility_Location')

Restricted license - for non-production use only - expires 2024-10-28


In [5]:
X=prob.addVars(num_customers,num_facilities, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=[f'fraction of demand from {sm[i]} serviced by {wh[j]}' for i in range (num_customers) for j in range (num_facilities)])

In [6]:
Y=prob.addVars(num_facilities,vtype=GRB.BINARY, name=[f'{wh[i]}' for i in range(num_facilities)])

In [7]:
prob.setObjective(sum(shipping_cost[i,j]*X[i,j] for i in range (num_customers) for j in range (num_facilities))+sum(setup_cost[i]*Y[i]for i in range(num_facilities)))

In [8]:
prob.addConstrs(sum(X[i,j] for j in range(num_facilities))==1 for i in range(num_customers))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
prob.addConstrs(X[i,j]<=Y[j] for i in range(num_customers) for j in range(num_facilities))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>}

In [10]:
prob.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5250U CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0xdd83b940
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Found heuristic solution: objective 6.0385165

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 1 nodes (15 simplex 

In [11]:
for v in prob.getVars():
    if v.x>0:
        print(f'{v.varName}={round(v.x,2)}')

fraction of demand fromsupermarket1 serviced by warehouse4=1.0
fraction of demand fromsupermarket2 serviced by warehouse4=1.0
warehouse4=1.0
